In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import time


 
usr = ""
pwd = ""
base_url = 'https://www.instagram.com/'

options = webdriver.ChromeOptions() 
options.add_argument("")

driver = webdriver.Chrome(executable_path='chromedriver', options=options)
driver.get(base_url)
driver.implicitly_wait(5)
 
#login
elem = driver.find_element_by_name("username")
elem.send_keys(usr)
elem = driver.find_element_by_name("password")
elem.send_keys(pwd)
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[2]/div[1]/div/form/div[4]').click()
driver.implicitly_wait(20)
time.sleep(5)



a_file = open("username.txt", "r")
username_list = []
stripped_line = ''

for line in a_file:
    stripped_line = line.strip()
    username_list.append(stripped_line)

    
a_file.close()
print(username_list)
 

f = open("usr_comment.csv", "w",encoding="utf-8")


#
for username in username_list:
    """With the input of an account page, scrape the 25 most recent posts urls"""
    url = "https://www.instagram.com/" + username + "/"
    post = 'https://www.instagram.com/p/'
    driver.get(url)
    post_links = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    time.sleep(2)

    while len(post_links) < 300:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break

        else:
            links = [a.get_attribute('href') for a in driver.find_elements_by_tag_name('a')]
            for link in links:
                if post in link and link not in post_links:
                    post_links.append(link)
        last_height = new_height
    print(post_links)
    print(len(post_links))
    
    #GET POST URL
    time.sleep(3)
    for link in post_links:
        url = link
        post = 'https://www.instagram.com/p/'
        driver.get(url)
        time.sleep(1.5)
        user_names = []
        user_comments = []
        comment = driver.find_elements_by_class_name('gElp9 ')
        
        # FOR COMMENT
        for c in comment:
            container = c.find_element_by_class_name('C4VMK')
            name = container.find_element_by_class_name('_6lAjh').text
            content = container.find_element_by_tag_name('span').text
            content = content.replace('\n', ' ').strip().rstrip()
            user_names.append(name)
            user_comments.append(content)

        print(user_names)
        print(user_comments)
        f.write(username+ ',' + url +','+ str(user_names)+ ',' + str(user_comments) + '\n')

# save
f.close()